In [ ]:
# config variables
from azure.identity import DefaultAzureCredential

credential=DefaultAzureCredential()
resource_group_name = "dantaylo_sdktest_rg3"
hub_name = "dantaylo_sdktest_hub3"
project_name = "dantaylo-sdktest-project3"
location = "eastus"
subscription_id = "597966d1-829f-417e-9950-8189061ec09c"
aiservices_resource_name = "dantaylo_sdktest_aiservices3"
aisearch_resource_name = "dantaylo-sdktest-aisearch3"
aisearch_index_name = "dantaylo_aisearch_index"
azure_openai_chat_model = "gpt-35-turbo"
azure_openai_embedding_model = "text-embedding-ada-002"

In [ ]:
# Use ML Client to set Azure OpenAI environment variables from the project connections
# NOTE: you can optionally write these out to a .env file

from azure.ai.ml import MLClient
import os

ml_client = MLClient(workspace_name=project_name, 
    resource_group_name=resource_group_name, 
    subscription_id=subscription_id, 
    credential=credential)

aiservices_connection = ml_client.connections.get("azure_aiservices_connection", populate_secrets=True)
os.environ['AZURE_OPENAI_ENDPOINT'] = aiservices_connection.target
os.environ['AZURE_OPENAI_KEY'] = aiservices_connection.api_key

In [ ]:
# Call OpenAI
from openai import AzureOpenAI

client = AzureOpenAI(
  azure_endpoint = os.environ['AZURE_OPENAI_ENDPOINT'],
  api_key=os.environ['AZURE_OPENAI_KEY'],
  api_version="2024-02-15-preview"
)

message_text = [{"role":"system","content":"You are an AI assistant that helps people find information."},
                {"role":"user","content":"Tell me about Banco Itaú"}]

completion = client.chat.completions.create(
  model=azure_openai_chat_model, messages = message_text, 
  temperature=0.7, max_tokens=800, top_p=0.95,
  frequency_penalty=0, presence_penalty=0, stop=None
)

print(completion.choices[0].message.content)

In [ ]:
from azure.ai.ml import MLClient
import os

# Use ML Client to set environment variables
ml_client = MLClient(workspace_name=project_name, 
    resource_group_name=resource_group_name, 
    subscription_id=subscription_id, 
    credential=credential)

aiservices_connection = ml_client.connections.get("azure_aisearch_connection", populate_secrets=True)
os.environ['AZURE_AISEARCH_ENDPOINT'] = aiservices_connection.target
os.environ['AZURE_AISEARCH_KEY'] = aiservices_connection.api_key
os.environ['AZURE_AISEARCH_INDEX_NAME'] = aisearch_index_name

In [ ]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery

index_name = os.environ["AZUREAI_SEARCH_INDEX_NAME"]
#  retrieve documents relevant to the user's question from Cognitive Search
search_client = SearchClient(
    endpoint=os.environ["AZURE_SEARCH_ENDPOINT"],
    credential=AzureKeyCredential(os.environ["AZURE_SEARCH_KEY"]),
    index_name=index_name)

aoai_client = AzureOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_KEY"],    
)

# generate a vector embedding of the user's question
embedding = aoai_client.embeddings.create(input=query,
    model=os.environ["AZURE_OPENAI_EMBEDDING_DEPLOYMENT"])
embedding_to_query = embedding.data[0].embedding

context = ""
# use the vector embedding to do a vector search on the index
vector_query = VectorizedQuery(vector=embedding_to_query, k_nearest_neighbors=num_docs, fields="contentVector")
results = search_client.search(
    search_text="", vector_queries=[vector_query], select=["id", "content"])

for result in results:
    context += f"\n### From: {result['id']}\n{result['content']}"




In [ ]:
import os
from azure.ai.contentsafety import ContentSafetyClient
from azure.core.credentials import AzureKeyCredential
from azure.core.exceptions import HttpResponseError
from azure.ai.contentsafety.models import AnalyzeTextOptions, TextCategory

text = "Your input text"

# analyze text
key = os.environ["AZURE_AISERVICES_KEY"]
endpoint = os.environ["AZURE_AISERVICES_ENDPOINT"]

# Create an Azure AI Content Safety client
client = ContentSafetyClient(endpoint, AzureKeyCredential(key))

# Contruct request
request = AnalyzeTextOptions(text="Your input text")

# Analyze text
try:
    response = client.analyze_text(request)
except HttpResponseError as e:
    print("Analyze text failed.")
    if e.error:
        print(f"Error code: {e.error.code}")
        print(f"Error message: {e.error.message}")
        raise
    print(e)
    raise

hate_result = next(item for item in response.categories_analysis if item.category == TextCategory.HATE)
self_harm_result = next(item for item in response.categories_analysis if item.category == TextCategory.SELF_HARM)
sexual_result = next(item for item in response.categories_analysis if item.category == TextCategory.SEXUAL)
violence_result = next(item for item in response.categories_analysis if item.category == TextCategory.VIOLENCE)

if hate_result:
    print(f"Hate severity: {hate_result.severity}")
if self_harm_result:
    print(f"SelfHarm severity: {self_harm_result.severity}")
if sexual_result:
    print(f"Sexual severity: {sexual_result.severity}")
if violence_result:
    print(f"Violence severity: {violence_result.severity}")
